# The CamVid Dataset

<!--- @wandbcode{sagemaker-studio-lab} -->

In this notebooks we will pull the Cambridge-driving Labeled Video Database or `CamVid` to train our model. It contains a collection of videos with object class semantic labels, complete with metadata. The database provides ground truth labels that associate each pixel with one of 32 semantic classes.

We will upload the full dataset to Weights and Biases as an `wandb.Artifact` first, and then compute some information of what classes are present on each image, and upload the processed dataset as a `wandb.Table`. Doing so enables the user to use the `wandb` UI to visualize and filter images.

In [ ]:
import wandb
from fastai.vision.all import *

Let's check that Weights and Biases is working and we are properly logged into our account

In [ ]:
# log to wandb
wandb.login()

## Log the raw dataset
We will grab a copy of `CamVid` using `fastai`'s `untar_data` method, afterwards we can use the `Artifact.add_dir()` method, and upload the full folder to our wandb workspace.

In [ ]:
path = untar_data(URLs.CAMVID)
codes = np.loadtxt(path/'codes.txt', dtype=str)
fnames = get_image_files(path/"images")
class_labels = {k: str(v) for k, v in enumerate(codes)}

In [ ]:
fnames

- we create a project under `user/project`
- If you are working on a team, you can pass the team name to `Entity`

In [ ]:
PROJECT="sagemaker_camvid_demo"
ENTITY=None

In [ ]:
with wandb.init(
    project=PROJECT,
    name="upload_camvid",
    entity=ENTITY,
    job_type="upload",
):
    artifact = wandb.Artifact(
        'camvid-dataset',
        type='dataset',
        metadata={
            "url": URLs.CAMVID,
            "class_labels": class_labels
        },
        description=("The Cambridge-driving Labeled Video Database (CamVid) is the first collection " 
                     "of videos with object class semantic labels, complete with metadata. " 
                     "The database provides ground truth labels that associate each pixel "
                     "with one of 32 semantic classes.")
    )
    artifact.add_dir(path)
    wandb.log_artifact(artifact)

## Log a `wandb.Table`
Let's log a `wandb.Table` with the frequency distribution of each class

![](images/camvid_table.png)

In [ ]:
one_image = fnames[0]
one_image

In [ ]:
path.ls()

In [ ]:
(path/"labels").ls()

In [ ]:
def label_func(fn):
    return fn.parent.parent/"labels"/f"{fn.stem}_P{fn.suffix}"

In [ ]:
label_func(one_image)

let's check that the mapping is working correctly

In [ ]:
show_images([load_image(one_image), load_image(label_func(one_image))], figsize=(15,8))

Let's contruct the frequency distribution

In [ ]:
mask = load_image(label_func(one_image))

In [ ]:
mask_np = np.array(mask)
mask_np

In [ ]:
np.unique(mask_np, return_counts=True)

let's count how many pixels we have per class

In [ ]:
def frequency_pixel_count(mask_data) -> dict:
    "Count pixels on the image for each class"
    (unique, counts) = map(list, np.unique(mask_data, return_counts=True))
    frequency_dict = {d:0 for d in class_labels.values()}
    for class_id, count in zip(unique, counts):
        frequency_dict[class_labels[class_id]] = count
    return frequency_dict

In [ ]:
frequency_pixel_count(mask_np)

In [ ]:
ARTIFACT_ID = 'capecape/sagemaker_camvid_demo/camvid-dataset:latest'

In [ ]:
## -1 to log full dataset
N_SAMPLES = 10

In [ ]:
def log_dataset():
    with wandb.init(
        project=PROJECT,
        name="visualize_camvid",
        entity=ENTITY,
        job_type="data_viz"
    ):
        artifact = wandb.use_artifact(ARTIFACT_ID, type='dataset')
        artifact_dir = artifact.download()

        labels = list(class_labels.values())
        
        # create an empty wandb.Table
        table = wandb.Table(columns=["File_Name", "Images"] + labels)
        
        # get all files
        image_files = get_image_files(Path(artifact_dir)/"images")[0:N_SAMPLES]
        
        print("Creating Table...")
        for image_file in progress_bar(image_files):
            image = Image.open(image_file)
            mask_data = np.array(Image.open(label_func(image_file)))
            # count of pixels per class
            pixel_count = frequency_pixel_count(mask_data)
            table.add_data(
                str(image_file.name),
                wandb.Image(image, masks={"predictions": {"mask_data": mask_data,
                                                          "class_labels": class_labels
                                                         }}),
                *pixel_count.values()
            )
        wandb.log({"CamVid_Dataset": table})

In [ ]:
log_dataset()

## View the dataset in Weights and Biases workspace

We get a nice UI to view our images

![](images/camvid_mask.gif)